<a href="https://colab.research.google.com/github/bintangnabiil/Machine_Learning/blob/main/Machine_Learning_UTS_RegresiUTSTelkom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. a) Pengumpulan dan Pembersihan Data (Pandas)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/RegresiUTSTelkom.csv')
print(df.head())
print(df.info())
print(df.isnull().sum())

df = df.drop_duplicates()
df.columns = ['feature_' + str(i) for i in range(df.shape[1])]

   2001  49.94357  21.47114   73.0775   8.74861  -17.40628  -13.09905  \
0  2001  48.73215  18.42930  70.32679  12.94636  -10.32437  -24.83777   
1  2001  50.95714  31.85602  55.81851  13.41693   -6.57898  -18.54940   
2  2001  48.24750  -1.89837  36.29772   2.58776    0.97170  -26.21683   
3  2001  50.97020  42.20998  67.09964   8.46791  -15.85279  -16.81409   
4  2001  50.54767   0.31568  92.35066  22.38696  -25.51870  -19.04928   

   -25.01202  -12.23257   7.83089  ...   13.0162  -54.40548  58.99367  \
0    8.76630   -0.92019  18.76548  ...   5.66812  -19.68073  33.04964   
1   -3.27872   -2.35035  16.07017  ...   3.03800   26.05866 -50.92779   
2    5.05097  -10.34124   3.55005  ...  34.57337 -171.70734 -16.96705   
3  -12.48207   -9.37636  12.63699  ...   9.92661  -55.95724  64.92712   
4   20.67345   -5.19943   3.63566  ...   6.59753  -50.69577  26.02574   

   15.37344   1.11144  -23.08793   68.40795  -1.82223  -27.46348   2.26327  
0  42.87836  -9.90378  -32.22788   70.49388  

##b) Feature Selection

In [ ]:
print(df.columns)

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
       'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',
       'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54',
       'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59',
       'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64',
       'f

###Constant dan Quasi-Constant Features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
selector.fit(df)
X_reduced = df.loc[:, selector.get_support()]

###Duplicate Features

In [ ]:
duplicated_features = []
for i in range(X_reduced.shape[1]):
    for j in range(i + 1, X_reduced.shape[1]):
        if X_reduced.iloc[:, i].equals(X_reduced.iloc[:, j]):
            duplicated_features.append(X_reduced.columns[j])
X_reduced = X_reduced.drop(columns=duplicated_features)

###Correlation (Pearson)

In [ ]:
import numpy as np

corr_matrix = X_reduced.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
X_reduced = X_reduced.drop(columns=to_drop)

###Mutual Information (MI)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# Asumsikan df sudah ada
X = df.drop(columns=['feature_0'])  # semua fitur selain kolom pertama
y = df['feature_0']  # kolom pertama sebagai target

mi = mutual_info_classif(X, y)
mi_series = pd.Series(mi, index=X.columns)
print(mi_series.sort_values(ascending=False))

feature_1     0.045688
feature_6     0.028188
feature_63    0.022025
feature_46    0.018462
feature_2     0.017217
                ...   
feature_42    0.002295
feature_30    0.002290
feature_45    0.002039
feature_65    0.001860
feature_77    0.001683
Length: 90, dtype: float64


##c) Feature Engineering

### Scaling (Transformasi Data)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

###Polynomial Features

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
import sklearn
print(sklearn.__version__)

1.6.1


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

# Misal X sudah ada (tanpa 'feature_0' karena itu target)
X = df.drop(columns=['feature_0'])
print(f"Awal jumlah fitur: {X.shape[1]}")

# 1️⃣ Seleksi fitur dulu: buang fitur yang variance-nya terlalu kecil
selector = VarianceThreshold(threshold=0.05)
X_selected = selector.fit_transform(X)
print(f"Jumlah fitur setelah seleksi variance: {X_selected.shape[1]}")

# 2️⃣ Kurangi fitur lagi kalau masih kebanyakan (misal pakai PCA ringan dulu)
if X_selected.shape[1] > 30:  # Misal kita mau max 30 fitur saja
    pca_reduction = PCA(n_components=30)
    X_selected = pca_reduction.fit_transform(X_selected)
    print(f"Jumlah fitur setelah pre-PCA: {X_selected.shape[1]}")

# 3️⃣ Sekarang buat PolynomialFeatures (degree=2, interaction_only)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_poly = poly.fit_transform(X_selected)
print(f"Jumlah fitur setelah PolynomialFeatures: {X_poly.shape[1]}")

# 4️⃣ Optional: Reduksi dimensi akhir supaya manageable
final_pca = PCA(n_components=50)
X_poly_pca = final_pca.fit_transform(X_poly)

# Wrap jadi DataFrame
X_poly_pca_df = pd.DataFrame(X_poly_pca, columns=[f'PC_{i+1}' for i in range(X_poly_pca.shape[1])])
print(f"Akhir jumlah fitur setelah PCA: {X_poly_pca_df.shape[1]}")
print(X_poly_pca_df.head())

Awal jumlah fitur: 90
Jumlah fitur setelah seleksi variance: 90
Jumlah fitur setelah pre-PCA: 30
Jumlah fitur setelah PolynomialFeatures: 465
Akhir jumlah fitur setelah PCA: 50
           PC_1           PC_2           PC_3           PC_4           PC_5  \
0 -1.453530e+06  721065.884315 -112053.337581  267568.866212 -246246.734310   
1 -2.159214e+04   -7207.420287   64057.958783   96098.103188 -137562.000679   
2 -1.394863e+06  769997.172507 -213827.417200 -343903.053693 -705187.105572   
3 -1.208183e+06 -111973.438731  176457.386508  180879.984739 -425308.759395   
4 -8.098500e+05  260021.477119 -163356.134715  214042.910966 -182090.565740   

            PC_6           PC_7           PC_8           PC_9          PC_10  \
0  107117.146677  -79405.356813  174100.450694 -182157.800604 -348359.176605   
1   46516.296330 -375159.439236  379739.874008  -66970.584246  290460.721390   
2  -75667.343888  -30632.943155  395194.189414 -152835.107962  294144.617001   
3 -215482.111523 -290875.889

##d) Model Training (Scikit-learn)

###Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_scaled, y)

LinearRegression()

###Polynomial Regression

In [ ]:
lr_poly = LinearRegression()
lr_poly.fit(X_poly, y)

LinearRegression()

###Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_scaled, y)

DecisionTreeRegressor()

###K-Neighbors Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn.fit(X_scaled, y)

KNeighborsRegressor()

###Bagging Regressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

bagging = BaggingRegressor()
bagging.fit(X_scaled, y)

BaggingRegressor()

###Boosting (AdaBoost)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# 1️⃣ Cek bentuk data
print(f"X_scaled shape: {X_scaled.shape}, y shape: {y.shape}")

# 2️⃣ Atur base estimator biar ringan
base_estimator = DecisionTreeRegressor(max_depth=3)

# 3️⃣ AdaBoost dengan estimator lebih sedikit dan base yang lebih kecil
ada = AdaBoostRegressor(
    estimator=base_estimator,  # kalau scikit-learn > 1.2
    n_estimators=30,  # lebih ringan dari default (50)
    learning_rate=0.5,  # percepat konvergensi
    random_state=42
)
ada.fit(X_scaled, y)

# 4️⃣ GradientBoosting: atur max_depth dan n_estimators supaya cepat
gbr = GradientBoostingRegressor(
    n_estimators=30,    # lebih ringan
    learning_rate=0.1,
    max_depth=3,        # kecilkan depth
    random_state=42
)
gbr.fit(X_scaled, y)

###Support Vector Regressor (SVR)

In [ ]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(X_scaled, y)